In [39]:
import numpy as np

In [40]:
def dphi(f,TECU):
    """
    given a frequency and integral on line-of-sight electons
    calulate phase shift
    """
    f=f*1e6 # from MHz to Hz
    phase=8.44e-7*1e16*TECU/f
    return phase

In [41]:
print dphi(1420,2), "Radian"
    

11.8873239437 Radian


# Ionospheric Pierce Point (IPP)

from a receiver at $(\lambda,\phi)$ pointing to source at (az,el) with a shell height H and earth radius R
for latitudes between 70S and 70N

$$\psi = \frac{\pi}{2} - el-arcsin( \frac{R}{R+H} cos(el))$$

$$sin(\phi_{ipp})  = sin(\phi)cos(\psi) + cos(\phi)sin(\psi)cos(az)$$

$$\lambda_{ipp} = \lambda + arcsin(\frac{sin(\psi)sin(az)}{cos(\phi_{ipp})})$$

In [42]:
lat=-30.711055555  # latitude of MeerKAT
lon=21.43388888    #longitude of MeerKAT
height=350.00 # height of layer above earth
r_e=6378.00 #km earth radius
factor=(r_e/(r_e+height))
print factor

0.947978596908


In [43]:
def psi(el,factor):
    psi=np.pi/2 -el -np.arcsin(factor*np.cos(el))
    return psi
    

In [44]:
print psi(0.9,factor)

0.040637674995325335


In [45]:
def lat_ipp(lat,psi,az):
    sin_ipp=np.sin(lat)*np.cos(psi)+np.cos(lat)*np.sin(psi)*np.cos(az)
    return np.arcsin(sin_ipp)

def lon_ipp(lon,psi,az,lat_ipp):
    """
    Assumes we are outside of polar regions
    which are above lat 70N or below lat 70S
    """
    l=lon+np.arcsin( (np.sin(psi)*np.sin(az))/np.cos(lat_ipp))
    return l


In [46]:
az=np.radians(90.)
la=np.radians(lat)
lo=np.radians(lon)
print az
for eldeg in np.arange(20,90,5):
    el=np.radians(eldeg)
    ps=psi(el,factor)
    lai=lat_ipp(la,ps,az)
    loi=lon_ipp(lo,ps,az,lai)
    print("elevation %6.3f psi %6.3f latitude %6.3f logitude %6.3f")\
    %(np.degrees(el), np.degrees(ps), np.degrees(lai), np.degrees(loi))
    #print np.degrees((el,ps,lai,loi))

1.5707963267948966
elevation 20.000 psi  7.025 latitude -30.456 logitude 29.590
elevation 25.000 psi  5.778 latitude -30.538 logitude 28.146
elevation 30.000 psi  4.818 latitude -30.591 logitude 27.033
elevation 35.000 psi  4.055 latitude -30.626 logitude 26.148
elevation 40.000 psi  3.432 latitude -30.650 logitude 25.424
elevation 45.000 psi  2.908 latitude -30.667 logitude 24.815
elevation 50.000 psi  2.458 latitude -30.680 logitude 24.292
elevation 55.000 psi  2.062 latitude -30.689 logitude 23.831
elevation 60.000 psi  1.706 latitude -30.696 logitude 23.418
elevation 65.000 psi  1.382 latitude -30.701 logitude 23.041
elevation 70.000 psi  1.081 latitude -30.705 logitude 22.691
elevation 75.000 psi  0.797 latitude -30.708 logitude 22.361
elevation 80.000 psi  0.525 latitude -30.710 logitude 22.045
elevation 85.000 psi  0.261 latitude -30.711 logitude 21.737


# From Effects Of The Ionosphere On Ground-Based Global 21 CM Signal Detection

$$\delta\theta = \frac{2d}{3R_E} ({\frac{\nu_p}{\nu}}^2) (1+ \frac{h_m}{R_E})({(sin^2(el) + \frac{2h_m}{R_E}})^{3/2} cos(el)$$


In [47]:
f_p=10 # MHz plasma frequency
# Usually in the range 1-10MHz
f=1420.4 #MHz
eldeg=20.0 #degrees
el=np.radians(eldeg)
R_E=6378.0 #km earth radius
h_m=300.0 #km height of F layer
d=200.0 #km thickness of ionosphere

print (f_p/f)**2,(1+h_m/R_E),np.sin(el)**2+ 2*h_m/R_E,np.cos(el)

def dtheta(R,d,f_p,f,h,eldeg):
    el=np.radians(eldeg)
    dt=(2*d)/(3*R)
    dt=dt*((f_p/f)**2)
    dt=dt*(1+h/R)
    dt=dt*(((np.sin(el)**2+ 2*h/R))**(1.5))
    dt=dt*np.cos(el)
    dt=np.degrees(dt)
    return dt

print 3600*dtheta(R_E,d,f_p,f,h_m,eldeg),"arcsec"

4.95654065501e-05 1.04703668862 0.2110511556747537 0.9396926207859084
0.020388597191271302 arcsec


# from Griday MSc thesis

$$\Delta\rho = \frac{40.308}{f^2}TEC$$

* $\Delta\rho$ is the change in path length from geometric path in m
* f in MHz
* vertial TEC is of order 6-20 (typically 10) higher in summer and lower in winter
* 1TEC=1e16m^2
* during solar flare up to 40!!
* also low at night (below 6) and high in day
* TEC is TECU along the sight line (of order 10/sin(elev))

# IERS technical note 36

$$\delta = -\frac{s_1}{f^2} -\frac{s_2}{f^4} - \frac{s_3}{f^f}$$
where $\theta$ is angle to earth's magnetic field

$s_1 = 40.309 \int{N_e dl}$

$s_2 = 1.1284\times 10^{12}\int{N_e B\cos(\theta)} $

$s_3 = 812.42\int{{N_e}^2 dl} + 1.5793\times 10^{22} \int{N_e B^2(1+\cos^2(\theta)) dl}$

all expessed in SI units

typically $B cos(\theta) \approx 10^4 nT$

In [48]:
TEC=5
f=151e6
print 40.308*1e16*TEC/f**2

88.39086005


In [49]:
def delay(f,TECU):
    """
    given a freq in MHz 
    and a slant (or vertical) TEC in TECU
    gives delay in metres
    """
    d=4.0308e5*TECU/(f**2)
    return d

print delay(1574.42,300)

48.7833149733


# from Ionosphere VLA 74MHz
 $$ \phi = 114\times\frac{74}{\nu}TEC$$
 * $\phi$ in radians
 * $\nu$ in MHz
 
 
 ## Another from Hemboldt
 $$ \phi = 84.36\times\frac{100}{\nu}TEC$$
 or
 $$ \phi = \frac{8436}{\nu}TEC$$
 or even
 $$ \phi = \frac{8.436}{\nu_{GHz}} TEC$$
 
 
 
 
 ## shift from refraction depends on difference in phase
 $$\theta = \frac{\lambda \Delta\phi}{2\pi B}$$
 
 * $\theta$ radians
 * B is basline in M
 * $\lambda$ wavelength
 

In [50]:
def refrac(freq,tec1,tec2):
    """
    freq in MHz
    differences in TEC should be mostly small (<<0.1)
    returns arcseconds
    """
    f74=freq/74.
    deltatec=tec1-tec2
    theta=deltatec/(6.95e-5*f74*f74)
    return theta

In [51]:
print refrac(1420.4,6.2,6.205)
print refrac(74,6.2,6.205)
"""
FYI a  median VLA 74MHz B array differential refraction across 5 degrees is about 25 arcsec at high elevations
and about 40arcec at elevations beow 40deg
see PROBING FINE-SCALE IONOSPHERIC STRUCTURE WITH THE VERY LARGE ARRAY RADIO TELESCOPE

tavelling TIDs have amplitudes of a few (1-5) percent
"""

-0.195266306668
-71.9424460432


'\nFYI a  median VLA 74MHz B array differential refraction across 5 degrees is about 25 arcsec at high elevations\nand about 40arcec at elevations beow 40deg\nsee PROBING FINE-SCALE IONOSPHERIC STRUCTURE WITH THE VERY LARGE ARRAY RADIO TELESCOPE\n\ntavelling TIDs have amplitudes of a few (1-5) percent\n'

# another version

$$\Delta\theta = -\frac{1}{8\pi^2}\frac{e^2}{\epsilon_0 m_e}\frac{1}{\nu^2}\nabla{STEC} [radian]$$

where STEC is the slant TEC 

In [52]:
def slant(elev,R=6378.0,H=350.0):
    """
    R=earth radius (km)
    H= Shell height (km)
    elev= elevation in deg
    """
    h=R/(R+H)
    x=np.cos(np.radians(elev))
    s=1./np.sqrt(1.0 - (h*x)**2)
    #print h
    #print x
    return s

In [53]:
# suppose zenith TEC is T and slant tec = T/sin(elev)
elev=20.0 #degrees

#better version for slant factor

T=6.0
print refrac(1712., T, T*slant(20,6378.0,350.0))

-193.68468331271734
